 # Classification des avis sur des vêtements de femmes vendus dans le e-commerce

- Est ce que les avis que l'on a des vêtements sont représentatifs de la note qui est attribuée ?

Idées :
- visualisation de données : quels types de vêtements ont les notes les plus élevées ?
- nb d'avis donné selon l'âge des clients

In [1]:
import pandas as pd
import spacy
import string
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer

## I. Import des données

Dans un premier temps, nous allons importer nos données. Notre base de données contient des informations sur des avis de vêtements femmes vendus sur internet. Ces données sont issus d'un processus de webscrapping.

In [2]:
# Import des données
data = pd.read_csv("Womens Clothing E-Commerce Reviews.csv", sep = ",")

# Renomage première colonne pour pouvoir l'utiliser comme id par la suite
data = data.rename(columns = {"Unnamed: 0" : "id"})

# Affichage des 5 premières lignes
data.head()

,id,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [4]:
data.shape

(23486, 11)

Notre jeu de données contient 23 486 avis et 11 colonnes. 

## II. Pré-traitement des données

Pour avoir des données plus propres, nous allons effectuer divers pré-traitements.

In [8]:
def preprocess_text(text):
    
    text = text.str.lower() # mise en minuscules du texte
    
    punct = string.punctuation
    text = ''.join([char for char in text if char not in punct]) # suppression de la ponctuation

    tokenizer = RegexpTokenizer('\w+')
    tokens = tokenizer.tokenize(text) # tokennisation du texte
    
    nlp = spacy.load('en_core_web_sm')
    # augmentation du nombre de caractères
    nlp.max_length = 6807745
    lemmatized_tokens = [token.lemma_ for token in nlp(' '.join(tokens))] # lemmatisation des tokens
    
    stemmer = SnowballStemmer('english')
    stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens] # racinisation des tokens
    
    processed_text = ' '.join(stemmed_tokens) # reconstitution du texte
    
    return processed_text

On convertit notre colonne 'Review Text' en chaîne de caractères pour pouvoir utiliser toutes les fonctions de pré-traitements.

In [9]:
data['Review Text'] = data['Review Text'].astype(str)

In [10]:
# Application de la fonction preprocess_text à la colonne 'Review Text' pour avoir des avis nettoyés
data['Review Text'] = preprocess_text(data['Review Text'])

On affiche notre dataframe pour vérifier que le pré-traitement soit bien réalisé.

In [1]:
data.head()

NameError: name 'data' is not defined

## III. Classification

### Traitement et séparation des données

Dans cette partie, nous allons chercher à classifier les avis en fonction de leur note. 

Nous allons utiliser la colonne "Rating" comme étiquettes et "Review Text" comme valeurs. 




#### Sélection des informations dans notre dataframe

Nous allons sélectionner les trois colonnes qui vont nous servir pour la classification dans un objectif d'optimiser les temps de calculs et de ne pas avoir d'informations superflus.

In [11]:
# On récupère la colonne id, Rating et Review Text
new_data = data[["id", "Rating", "Review Text"]]
new_data.head()

,id,Rating,Review Text
2,2,3,I had such high hopes for this dress and reall...
3,3,5,"I love, love, love this jumpsuit. it's fun, fl..."
4,4,5,This shirt is very flattering to all due to th...
5,5,2,"I love tracy reese dresses, but this one is no..."
6,6,5,I aded this in my basket at hte last mintue to...


#### Suppression des valeurs manquantes

In [ ]:
new_data = new_data.dropna() # On supprime les lignes avec des valeurs manquantes

In [ ]:
data.shape, new_data.shape

Suite à cette manipulation, nous avons 

#### Analyse de la colonne "Rating"

In [12]:
# Analyse de la colonne "Rating"
data["Rating"].value_counts()

Rating
5    10858
4     4289
3     2464
2     1360
1      691
Name: count, dtype: int64

Nous avons ici des notes allant de 1 à 5. Nous allons diviser ces valeurs en 3 catégories : 
- -1 pour les notes allant de 1 à 2
- 0 pour les notes égales à 3 
- 1 pour les plus élevées (4 et 5)

#### Analyse de la colonne "Review Text"

Notre colonne correspondant aux valeurs est "Review Text". \
Cette colonne contient tous les avis laissés par les internautes sur les différents vêtements.

#### Changement des étiquettes

Pour réaliser notre classification, nous allons donc modifier les étiquettes comme précisé ci-dessus. 

In [13]:
def map_label_to_numeric(label):
    return 1 if label == 5 else 0 if label == 3 or label == 4 else -1

In [14]:
def get_labels(data):
    labels = data[["id","Rating"]]
    labels['Rating'] = labels['Rating'].apply(map_label_to_numeric)
    labels.set_index('id', inplace=True)
    
    # ajouter les labels dans data selon l'id
    data['score_avis'] = labels

    # data['score_avis'] = labels
    return data

In [15]:
data = get_labels(data)
data.head()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_19012\29256686.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels['Rating'] = labels['Rating'].apply(map_label_to_numeric)


,id,Rating,Review Text,score_avis
2,2,3,I had such high hopes for this dress and reall...,0
3,3,5,"I love, love, love this jumpsuit. it's fun, fl...",1
4,4,5,This shirt is very flattering to all due to th...,1
5,5,2,"I love tracy reese dresses, but this one is no...",-1
6,6,5,I aded this in my basket at hte last mintue to...,1


In [16]:
# On analyse la nouvelle colonne "score_avis"
data["score_avis"].value_counts()

score_avis
 1    10858
 0     6753
-1     2051
Name: count, dtype: int64

Grâce à cette manipulation, nous pouvons observer que les avis ayant la note de 5 sont majoritaires dans notre jeu de données puisque cela correspond à la note de 1. Les avis compris entre 1 et 2 ont une proportion plus faible (-1). 

A présent, nous n'avons plus besoin de la colonne Rating, nous pouvons donc la supprimer du dataframe.

In [17]:
data = data[["id", "Review Text", "score_avis"]]
data.head()

,id,Review Text,score_avis
2,2,I had such high hopes for this dress and reall...,0
3,3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,4,This shirt is very flattering to all due to th...,1
5,5,"I love tracy reese dresses, but this one is no...",-1
6,6,I aded this in my basket at hte last mintue to...,1


#### Division de notre dataframe

Pour réaliser notre classification, nous avons besoin de séparer notre jeu de données en un jeu d'apprentissage, de validation et de test.

Note :
Les données en apprentissage automatique sont généralement séparées en trois jeux :
+ **entraînement** : données destinées à l'apprentissage du modèle ;
+ **validation** : données destinées à une évaluation intermédiaire du modèle pour permettre l'ajustement de ses hyperparamètres. Une fois les hyperparamètres du modèle arrêtés, on peut le ré-entraîner sur l'ensemble des données (entraînement + validation) avant de le tester sur le jeu de test ;
+ **test** : données destinées EXCLUSIVEMENT à l'évaluation FINALE (à réaliser une fois uniquement !) du modèle choisi finalement. Elles ne doivent sous aucune forme servir à la conception du modèle. Il est donc interdit aussi bien de les examiner que d'évaluer le modèle en cours de développement sur ce jeu de données.

Pour créer l'ensemble de validation, nous allons effectuer la manipulation à la fin du pré-traitement réalisé lors de la classification. 

In [18]:
# Fonction pour diviser de notre jeu de données en 2 : train et test
def split_data(data, train_ratio):
    data_train = data.sample(frac = train_ratio)
    data_test = data.drop(data_train.index)
    return data_train, data_test

# Diviser notre jeu de données en 2 : train et test
data_train, data_test = split_data(data, 0.6)

In [19]:
data_train.shape, data_test.shape

((11797, 3), (7865, 3))

Dans notre cas :
+ entraînement (appelé *Train*) contenant 11797 observations ;
+ validation (appelé *Validation*) contenant 5243 observations ;
+ test (appelé *Test*), contenant 2622 observations, soit environ 22% de la taille du jeu d'entraînement.

In [20]:
data_train.head()

,id,Review Text,score_avis
14207,14207,This tank/blouse is flowy and i love the desig...,1
20479,20479,This is a really cute style of t-sihrt. i tota...,0
18930,18930,"These are not only comfortable, but so cute. t...",1
20736,20736,I love these pants. they are very flattering a...,1
16451,16451,I bought this well made cute top today and lov...,1


In [21]:
data_test.head()

,id,Review Text,score_avis
4,4,This shirt is very flattering to all due to th...,1
5,5,"I love tracy reese dresses, but this one is no...",-1
7,7,"I ordered this in carbon for store pick up, an...",0
12,12,More and more i find myself reliant on the rev...,1
13,13,Bought the black xs to go under the larkspur m...,1


### Exploration des données

#### Distribution des classes

Il est important de connaître la répartition des classes dans les données d'entraînement pour pouvoir procéder à notre classification.

In [22]:
# Analyse de la colonne "score_avis" de notre jeu de données d'entrainement
print(data_train["score_avis"].value_counts())

# Calcul des proportions de chaque classe dans notre jeu de données d'entrainement
data_train["score_avis"].value_counts()/len(data_train)

score_avis
 1    6546
 0    4024
-1    1227
Name: count, dtype: int64


score_avis
 1    0.554887
 0    0.341104
-1    0.104009
Name: count, dtype: float64

Nous pouvons observer que les classes de scores sont réparties de manière aléatoire dans notre jeu d'apprentissage. Nous pouvons noter plus de 50% d'avis très favorables, correspondant à la note de 5/5. Les avis négatifs sont en minorité dans notre jeu d'entraînement.

#### Exploration du texte

Pour se faire une idée des textes auxquels nous avons affaire, nous allons les afficher pour savoir quels pré-traitements sont nécessaires.

In [23]:
# Affichage des 5 premiers avis
data_train["Review Text"].values[:5]

array(['This tank/blouse is flowy and i love the design. it was immediately comfortable and seemed true to size. i wore it to a family event with a sweater, black leggings and boots. i\'m 5\'5," 36d and usually wear medium in retailer tops and this fit beautifully from the get go, just love it!',
       'This is a really cute style of t-sihrt. i totally disagree with the reviewers who thought this was too short in the torso and/or overwhelming. i\'m 5\'8" and about 145 pounds and this fits just fine. (i ordered a size small.) beautiful drape and (i thought) relatively slim cut. the only reason i\'m not giving this 5 stars across the board is that the fabric is a little too thin (not sheer or see-through, but not as substantial as i would expect for the price.',
       'These are not only comfortable, but so cute. they are something you don\'t see on everyone else, and people always comment. i get so many compliments or inquiries every time i have them on. the materiel is lightweight an

## Exploration des données

In [24]:
from collections import Counter
import pandas as pd

class_distribution = (pd.DataFrame.from_dict(Counter(data_train.score_avis.values),
                                             orient='index')
                                  .rename(columns={0: 'num_examples'}))
class_distribution.index.name = 'class'
class_distribution

,num_examples
class,
1,6546
0,4024
-1,1227


In [25]:
import numpy as np

class_distribution['perc_examples'] = np.around(class_distribution.num_examples /
                                                np.sum(class_distribution.num_examples),
                                                2)
class_distribution

,num_examples,perc_examples
class,,
1,6546,0.55
0,4024,0.34
-1,1227,0.10


## Représentation des textes

### Sélection de descripteurs : prétraitements textuels

#### Exemple sur un avis

In [26]:
tw = data_train['Review Text'].iloc[100]
tw

"Absolutely beautiful and well made. this pair of jeans is perfect, no flaws whatsoever. i usually wear size 27 but bought the 26 as it fits better. 27 is very slightly loose but i think it supposed to be like that for a boyfriend jeans. i prefer it nice and fitted, so 26 works better. i'm lucky to find them still available in the store while it's not online. i didn't hesitate to buy at full price, which i rarely did. i'm so in love with my new purchase. very happy!"

Pour effectuer nos tâches de traitements de langage, nous allons transformer le 1er avis en utilisant spacy. En effet, spacy a plusieurs fonctionnalités : 
- permet de tokeniser directement notre texte
- peut lemmatiser les mots
- identifier et classer les entités nommées
- analyser les dépendances syntaxiques entre les mots
- représenter les mots sous forme de vecteurs (embedding)
- etc

In [27]:
# Pour la langue anglaise
!pip install -U spacy
! python -m spacy validate

import spacy
!python -m spacy download en_core_web_sm

nlp = spacy.load('en_core_web_sm')


⠙ Loading compatibility table...
⠹ Loading compatibility table...
⠸ Loading compatibility table...
✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.7.2) =================
ℹ spaCy installation:
c:\Users\ASUS\anaconda3\Lib\site-packages\spacy

NAME              SPACY            VERSION                            
en_core_web_sm    >=3.7.2,<3.8.0   3.7.1   ✔
fr_core_news_sm   >=3.7.0,<3.8.0   3.7.0   ✔

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 320.0 kB/s eta 0:00:40
     --------------------------------------- 0.0/12.8 MB 487.6 kB/s eta 0:00:27
      --------------------------------------- 0.2/12.8 MB 1.2 MB/s eta 0:00:11
     -- ------------------------------------- 0.8/12.8 MB 4.4 MB/s eta 0:00:03
     -------- ------------------------------- 2.7/12.8 MB 11.5 MB/s eta 0:00:01
     ------------------- -------------------- 6.2/12.8 MB 21.9 MB/s eta 0:00:01
     ---

In [28]:
# Transformation du premier avis en objet spacy
avis_nlp = nlp(avis) 
avis_nlp

This dress in a lovely platinum is feminine and fits perfectly, easy to wear and comfy, too! highly recommend!

In [29]:
type(avis_nlp)

spacy.tokens.doc.Doc

On a bien un objet de type spacy.

On affiche chaque token de notre objet spacy :

In [30]:
for token in avis_nlp:
    print(token)

This
dress
in
a
lovely
platinum
is
feminine
and
fits
perfectly
,
easy
to
wear
and
comfy
,
too
!
highly
recommend
!


Ensuite, nous allons pouvoir afficher les lemmes de chaque token. Grâce à cette étape, nous allons pouvoir simplifier les mots pour faciliter notre analyse textuelle.

In [31]:
# Lemmatisation
for token in avis_nlp:
    print(token.lemma_)

this
dress
in
a
lovely
platinum
be
feminine
and
fit
perfectly
,
easy
to
wear
and
comfy
,
too
!
highly
recommend
!


#### Généralisation de la lemmatisation


Pour lemmatiser notre texte, nous allons définir une fonction. Cette étape est indispensable pour récupérer les lemmes des mots de notre texte d'origine. Nous allons simplifier notre texte grâce à cette fonction.

Cette fonction va nous permettre de généraliser par la suite nos manipulations.

In [32]:
def lemmatise_text(text):
    text = nlp(text) # on transforme le texte en objet spacy
    lemmas = [token.lemma_ for token in text] # on récupère les lemmes
    return ' '.join(lemmas) # on retourne les lemmes sous forme de texte

In [33]:
# On teste sur 3 avis 
for avis in liste_avis[:3]:
    print("Avis initial : ", avis) # On affiche l'avis initial
    print("Avis lemmatisé : ", lemmatise_text(avis)) # On applique la fonction à notre avis

Avis initial :  I had such high hopes for this dress and really wanted it to work for me. i initially ordered the petite small (my usual size) but i found this to be outrageously small. so small in fact that i could not zip it up! i reordered it in petite medium, which was just ok. overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers. imo, a major design flaw was the net over layer sewn directly into the zipper - it c
Avis lemmatisé :  I have such high hope for this dress and really want it to work for I . I initially order the petite small ( my usual size ) but I find this to be outrageously small . so small in fact that I could not zip it up ! I reorder it in petite medium , which be just ok . overall , the top half be comfortable and fit nicely , but the bottom half have a very tight under layer and several somewhat cheap ( net ) over layer . imo , a major design flaw be the net over layer

Nous allons ensuite ajouter une colonne avec la fonction de **lemmatisation** appliquée à nos avis.

In [34]:
data_train['lemmas'] = data_train['Review Text'].apply(lemmatise_text)

In [35]:
data_train.head()

,id,Review Text,score_avis,lemmas
14207,14207,This tank/blouse is flowy and i love the desig...,1,this tank / blouse be flowy and I love the des...
20479,20479,This is a really cute style of t-sihrt. i tota...,0,this be a really cute style of t - sihrt . I t...
18930,18930,"These are not only comfortable, but so cute. t...",1,"these be not only comfortable , but so cute . ..."
20736,20736,I love these pants. they are very flattering a...,1,I love these pant . they be very flattering an...
16451,16451,I bought this well made cute top today and lov...,1,I buy this well make cute top today and love i...


On effectue la même manipulation sur l'ensemble de test .

In [36]:
data_test['lemmas'] = data_test['Review Text'].apply(lemmatise_text)

In [37]:
data_test.shape

(7865, 4)

In [38]:
# Sauvegarde
data_train.to_pickle('train.pkl')
data_test.to_pickle('test.pkl')

##### Racines

Pour réduire les mots à leur forme de base, nous allons utiliser SnowballStemmer sur notre texte. Pour cela, nous allons créer une fonction et l'appliquer à nos avis. 

In [39]:
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

In [40]:
def stem_text(text):
    stemmer = SnowballStemmer('english')
    tokenizer = RegexpTokenizer('\w+')
    stems = [stemmer.stem(token) for token in tokenizer.tokenize(text)]
    return ' '.join(stems)

In [41]:
stem_text(avis)

'this shirt is veri flatter to all due to the adjust front tie it is the perfect length to wear with leg and it is sleeveless so it pair well with ani cardigan love this shirt'

On applique la fonction à notre dataframe.

In [42]:
data_train['stems'] = data_train['Review Text'].apply(stem_text)

In [43]:
data_train.head()

,id,Review Text,score_avis,lemmas,stems
14207,14207,This tank/blouse is flowy and i love the desig...,1,this tank / blouse be flowy and I love the des...,this tank blous is flowi and i love the design...
20479,20479,This is a really cute style of t-sihrt. i tota...,0,this be a really cute style of t - sihrt . I t...,this is a realli cute style of t sihrt i total...
18930,18930,"These are not only comfortable, but so cute. t...",1,"these be not only comfortable , but so cute . ...",these are not onli comfort but so cute they ar...
20736,20736,I love these pants. they are very flattering a...,1,I love these pant . they be very flattering an...,i love these pant they are veri flatter and co...
16451,16451,I bought this well made cute top today and lov...,1,I buy this well make cute top today and love i...,i bought this well made cute top today and lov...


On fait de même sur l'ensemble de test.

In [44]:
data_test['stems'] = data_test['Review Text'].apply(stem_text)

In [45]:
data_test.shape

(7865, 5)

In [46]:
# Sauvegarde
data_train.to_pickle('train.pkl')
data_test.to_pickle('test.pkl')

##### Étiquettes morphosyntaxiques

Puis, nous allons analyser notre texte et renvoyer chaque mot remplacé par sa catégorie grammaticale pour continuer l'étude des avis.

In [47]:
def replace_words_with_pos_tag(text):
    text = nlp(text)
    return ' '.join([token.pos_ for token in text])

On teste la fonction sur un avis.

In [48]:
replace_words_with_pos_tag(avis)

'DET NOUN AUX ADV ADJ ADP PRON ADP ADP DET ADJ ADJ NOUN PUNCT PRON AUX DET ADJ NOUN PART VERB ADP NOUN CCONJ PRON AUX ADJ SCONJ PRON VERB ADV ADP DET NOUN PUNCT VERB DET NOUN PUNCT PUNCT PUNCT'

On applique la fonction à notre ensemble d'entrainement et on ajoute une colonne à notre dataframe.

In [49]:
data_train['pos'] = data_train['Review Text'].apply(replace_words_with_pos_tag)

In [50]:
data_train.head()

,id,Review Text,score_avis,lemmas,stems,pos
14207,14207,This tank/blouse is flowy and i love the desig...,1,this tank / blouse be flowy and I love the des...,this tank blous is flowi and i love the design...,DET NOUN SYM NOUN AUX ADJ CCONJ PRON VERB DET ...
20479,20479,This is a really cute style of t-sihrt. i tota...,0,this be a really cute style of t - sihrt . I t...,this is a realli cute style of t sihrt i total...,PRON AUX DET ADV ADJ NOUN ADP PROPN PUNCT NOUN...
18930,18930,"These are not only comfortable, but so cute. t...",1,"these be not only comfortable , but so cute . ...",these are not onli comfort but so cute they ar...,PRON AUX PART ADV ADJ PUNCT CCONJ ADV ADJ PUNC...
20736,20736,I love these pants. they are very flattering a...,1,I love these pant . they be very flattering an...,i love these pant they are veri flatter and co...,PRON VERB DET NOUN PUNCT PRON AUX ADV ADJ CCON...
16451,16451,I bought this well made cute top today and lov...,1,I buy this well make cute top today and love i...,i bought this well made cute top today and lov...,PRON VERB PRON ADV VERB ADJ NOUN NOUN CCONJ VE...


On effectue la même manipulation sur notre ensemble de test.

In [51]:
data_test['pos'] = data_test['Review Text'].apply(replace_words_with_pos_tag)

In [52]:
data_test.shape

(7865, 6)

In [53]:
# Sauvegarde
data_train.to_pickle('train.pkl')
data_test.to_pickle('test.pkl')

#####  Classe d'appartenance des entités nommées

Pour pouvoir effectuer la reconnaissances d'entités nommées sur nos avis, nous allons retourner une version de notre avis ou chaque entité nommée est remplacée par son type d'entité.

In [54]:
def ner(text):

    text = nlp(text) # on transforme le texte en objet spacy
    
    new_text = [] # on crée une liste vide

    for token in text: # pour chaque token dans l'avis

        # print(token.text, token.ent_iob_, token.ent_type_)
        
        if token.ent_iob_ == "O": # si l'entité ne fait pas partie d'une entité nommée
            new_text.append(token.text) # on ajoute le texte du token à la liste
        elif token.ent_iob_ == "B": # si l'entité fait partie d'une entité nommée
            new_text.append(token.ent_type_) # on ajoute le type de l'entité à la liste

        # Si l'entité comprend plusieurs mot on ne répète pas l'étiquette
        else:
            continue
    return ' '.join(new_text) # on retourne les étiquettes sous forme de texte

On applique la fonction sur trois avis.

In [55]:
# On applique la fonction sur trois avis.
for avis in liste_avis[:3]:
    print("Avis initial : ", avis) # On affiche l'avis initial
    print("Avis avec entités nommées : ", ner(avis)) # On applique la fonction à notre avis

Avis initiaux :  This shirt is very flattering to all due to the adjustable front tie. it is the perfect length to wear with leggings and it is sleeveless so it pairs well with any cardigan. love this shirt!!!
Avec les entités nommées :  This shirt is very flattering to all due to the adjustable front tie . it is the perfect length to wear with leggings and it is sleeveless so it pairs well with any cardigan . love this shirt ! ! !


Ici, l'utilisation d'entitées nommées ne nous apporte pas d'information puisqu'il s'agit d'avis. On rapelle qu'un avis ne contient pas forcément d'avis, de lieux, de dates etc.

###  Calcul des valeurs des descripteurs

Pour procéder aux calculs, nous allons séparer notre jeu de données d'entraînement pour avoir un jeu de données de validation. Les données test nou servirons pour l'évaluation finale des modèles. 

In [61]:
from sklearn.model_selection import train_test_split

In [62]:
X_train, X_valid, y_train, y_valid = train_test_split(data_train['Review Text'],
                                                      data_train['score_avis'],
                                                      train_size=0.75,
                                                      random_state=5)

In [63]:
X_train.shape, X_valid.shape

((8847,), (2950,))

On a donc 8 847 lignes dans notre jeu d'entrainement et 2 950 dans celui de validation.

In [64]:
y_train

13907    0
16335    1
20432    0
2500     0
1540     0
        ..
12151    0
14584    1
23297    1
2481     1
10811    1
Name: score_avis, Length: 8847, dtype: int64

Nous pouvons observer que les sorties à prédire correspondent aux trois étiquettes que nous avons défini plus haut.

Pour évaluer notre modèle, nous initialisons les ensembles de test.

In [65]:
# On récupère les avis et les labels du jeu de données de test
X_test, y_test = data_test['Review Text'], data_test['score_avis'] 

### Binaire : présence/absence

In [66]:
from sklearn.feature_extraction.text import CountVectorizer

bin_count = CountVectorizer(binary=True)

In [67]:
bin_count.fit(X_train)
bin_count

CountVectorizer(binary=True)

In [68]:
X_train_vectorized_bin = bin_count.transform(X_train)
X_train_vectorized_bin

<8847x9562 sparse matrix of type '<class 'numpy.int64'>'
	with 387246 stored elements in Compressed Sparse Row format>

In [69]:
X_valid_vectorized_bin = bin_count.transform(X_valid)
X_test_vectorized_bin = bin_count.transform(X_test)

In [70]:
X_valid_vectorized_bin # MEME NOMBRE DE COLONNES QUE X_train_vectorized_bin

<2950x9562 sparse matrix of type '<class 'numpy.int64'>'
	with 128942 stored elements in Compressed Sparse Row format>

###  Numérique discret : décomptes d'occurrence

Nous allons calculer les fréquences d'occurence des termes dans nos avis. 

In [71]:
vect_count = CountVectorizer().fit(X_train)

Nous pouvons examiner le vocabulaire de nos avis : 

In [72]:
vect_count.get_feature_names_out()[:50] # 50 premiers mots ("types" du vocabulaire)

array(['00', '00p', '02', '03', '0in', '0p', '0petite', '0r', '0verall',
       '0xs', '10', '100', '1000', '100lbs', '102', '102lbs', '103',
       '103lb', '103lbs', '104', '104lbs', '105', '105lb', '105lbs',
       '106', '106lbs', '107', '107lb', '107lbs', '107pound', '108',
       '108lbs', '109', '109lbs', '10lbs', '10p', '10x', '11', '110',
       '110lbs', '111lbs', '112', '112lb', '112lbs', '112llbs', '113',
       '113lbs', '114', '114lbs', '115'], dtype=object)

In [73]:
vect_count.get_feature_names_out()[-50:] # 50 derniers mots ("types" du vocabulaire)

array(['yes', 'yesterday', 'yet', 'yey', 'yfit', 'yield', 'yielded',
       'yikes', 'yo', 'yoga', 'yogini', 'yoke', 'york', 'you', 'young',
       'younger', 'your', 'youre', 'yourself', 'yourselves', 'youthful',
       'yr', 'yrs', 'yuck', 'yucky', 'yummiest', 'yummy', 'yup', 'zag',
       'zermatt', 'zero', 'zeros', 'zig', 'zigzag', 'zigzagging',
       'zillion', 'zip', 'zipepr', 'ziploc', 'zipped', 'zipper',
       'zippered', 'zippers', 'zippie', 'zipping', 'zips', 'zombie',
       'zone', 'zoom', 'zuma'], dtype=object)

Taille de notre vocabulaire :

In [74]:
len(vect_count.get_feature_names_out()) 

9562

#### Création matrice document-termes

Nous allons créer la matrice document-termes avec le même vectoriseur.

In [75]:
X_train_vectorized_count = vect_count.transform(X_train)
X_train_vectorized_count

<8847x9562 sparse matrix of type '<class 'numpy.int64'>'
	with 387246 stored elements in Compressed Sparse Row format>

In [76]:
X_valid_vectorized_count = vect_count.transform(X_valid)
X_test_vectorized_count = vect_count.transform(X_test)

A présent, nous allons prendre en compte les bi-grammes dans notre vocabulaire. 

In [77]:
vect_count_bigrams = CountVectorizer(min_df=5, ngram_range=(1,2)).fit(X_train)
X_train_vectorized_count_bigrams = vect_count_bigrams.transform(X_train)
X_valid_vectorized_count_bigrams = vect_count_bigrams.transform(X_valid)
X_test_vectorized_count_bigrams = vect_count_bigrams.transform(X_test)

In [78]:
len(vect_count_bigrams.get_feature_names_out())

17352

Nous avons presque 2 fois plus de vocabulaire avec inclusion des bigrammes.

# TRI-GRAMMES 
# filtres sur des catégories (adj+nom)

###  Numérique continu : TF-IDF (ou autres pondérations)

Nous allons limiter le vocabulaire à des termes qui apparaissent au moins 5 fois dans le document.

In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [80]:
vect_tfidf = TfidfVectorizer(min_df=5).fit(X_train)

In [81]:
len(vect_count.get_feature_names_out()), len(vect_tfidf.get_feature_names_out())

(9562, 3231)

La réduction de la taille du vocadulaire est importante et est due au paramètre min_df=5 : on a quasiment 3 fois moins de termes !

Nous allons vectoriser les jeux de données. 

In [82]:
# Vectorisation des corpus d'entrainement, de validation et de test
X_train_vectorized_tfidf = vect_tfidf.transform(X_train)
X_valid_vectorized_tfidf = vect_tfidf.transform(X_valid)
X_test_vectorized_tfidf = vect_tfidf.transform(X_test)

## Classification des textes

Nous allons réaliser une classification en utilisant plusieurs modèles afin de comparer les performances. 

In [83]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

### Modèles de référence faibles

#### Choix aléatoire

Nous allons d'abord choisir un modèle où toutes les classes ont la même probabilité d'être choisies ou bien le prédicteur respecte la disctribution des classes dans les données d'entrainement.

In [84]:
from sklearn.dummy import DummyClassifier

Prédiction proportionnelle à la distribution des classes dans les données d'entraînement :

In [85]:
random_prop_class = DummyClassifier(strategy='stratified').fit(X_train_vectorized_tfidf,
                                                               y_train)
predictions_valid = random_prop_class.predict(X_valid_vectorized_tfidf)
conf_mat = confusion_matrix(y_valid, predictions_valid)

In [86]:
print(conf_mat)

[[ 37 105 170]
 [107 324 588]
 [173 539 907]]


Prédiction uniforme : 

In [87]:
random_uniform = DummyClassifier(strategy='uniform').fit(X_train_vectorized_tfidf,
                                                         y_train)
predictions_valid = random_uniform.predict(X_valid_vectorized_tfidf)
predictions_valid

array([ 1,  1,  0, ...,  1,  0, -1], dtype=int64)

In [88]:
conf_mat = confusion_matrix(y_valid, predictions_valid)

In [89]:
print(conf_mat)

[[100 109 103]
 [358 336 325]
 [547 563 509]]


In [90]:
accuracy_score(y_valid, predictions_valid)

0.32033898305084746

In [91]:
print(classification_report(y_valid, predictions_valid))

              precision    recall  f1-score   support

          -1       0.10      0.32      0.15       312
           0       0.33      0.33      0.33      1019
           1       0.54      0.31      0.40      1619

    accuracy                           0.32      2950
   macro avg       0.33      0.32      0.29      2950
weighted avg       0.42      0.32      0.35      2950



#### Prédiction constante de la classe majoritaire

Nous allons d'abord identifier la répartition des classes dans les données d'entrainement.

In [92]:
class_distribution

,num_examples,perc_examples
class,,
1,6546,0.55
0,4024,0.34
-1,1227,0.10


In [93]:
maj = DummyClassifier(strategy='most_frequent').fit(X_train_vectorized_tfidf, y_train)
predictions_valid = maj.predict(X_valid_vectorized_tfidf)
predictions_valid

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [94]:
import numpy as np 

maj_class = (class_distribution.index[class_distribution.perc_examples ==
                                      np.amax(class_distribution.perc_examples)][0])
maj_class

1

In [95]:
np.all(predictions_valid == maj_class)

True

In [96]:
maj.score(X_valid_vectorized_tfidf, y_valid)

0.5488135593220339

In [97]:
print(classification_report(y_valid, predictions_valid))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       312
           0       0.00      0.00      0.00      1019
           1       0.55      1.00      0.71      1619

    accuracy                           0.55      2950
   macro avg       0.18      0.33      0.24      2950
weighted avg       0.30      0.55      0.39      2950



c:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Classifieur naïf bayésien

In [98]:
from sklearn.naive_bayes import MultinomialNB

In [99]:
model_nb = MultinomialNB().fit(X_train_vectorized_tfidf, y_train)
predictions_valid = model_nb.predict(X_valid_vectorized_tfidf)

In [100]:
accuracy_score(y_valid, predictions_valid)

0.6749152542372882

In [101]:
print(classification_report(y_valid, predictions_valid))

              precision    recall  f1-score   support

          -1       0.80      0.01      0.03       312
           0       0.58      0.46      0.51      1019
           1       0.71      0.94      0.81      1619

    accuracy                           0.67      2950
   macro avg       0.70      0.47      0.45      2950
weighted avg       0.67      0.67      0.62      2950



### Régression logistique

In [102]:
from sklearn.linear_model import LogisticRegression

In [103]:
model_lr = LogisticRegression(multi_class='multinomial', solver='lbfgs',
                              max_iter=200).fit(X_train_vectorized_count, y_train)

c:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [104]:
predictions_valid = model_lr.predict(X_valid_vectorized_count)

In [105]:
accuracy_score(y_valid, predictions_valid)

0.6850847457627118

In [106]:
print(classification_report(y_valid, predictions_valid))

              precision    recall  f1-score   support

          -1       0.48      0.41      0.44       312
           0       0.57      0.56      0.57      1019
           1       0.78      0.82      0.80      1619

    accuracy                           0.69      2950
   macro avg       0.61      0.59      0.60      2950
weighted avg       0.68      0.69      0.68      2950



In [107]:
def print_n_strongly_associated_features(vectoriser, model, n):
    feature_names = np.array(vectoriser.get_feature_names_out())

    for i in range(3):
        class_name = model.classes_[i]
        print("CLASSE {}".format(class_name))
        idx_coefs_sorted = model.coef_[i].argsort() # ordre croissant
        print("Les dix variables ayant l'association négative la plus forte " + 
              "avec la classe {} :\n{}\n".format(class_name,
                                                 feature_names[idx_coefs_sorted[:n]]))
        idx_coefs_sorted = idx_coefs_sorted[::-1] # ordre décroissant
        print("Les dix variables ayant l'association positive la plus forte " +
              "avec la classe {} :\n{}\n"
              .format(class_name,
                      feature_names[idx_coefs_sorted[:n]]))
        print()

Examinons les variables (termes) ayant l'association la plus forte avec chaque classe.

In [108]:
print_n_strongly_associated_features(vect_count, model_lr, 10)

CLASSE -1
Les dix variables ayant l'association négative la plus forte avec la classe -1 :
['amazing' 'compliments' 'wait' 'meant' 'worried' 'stunning' 'threads'
 'versatile' 'fan' 'avoid']

Les dix variables ayant l'association positive la plus forte avec la classe -1 :
['horrible' 'poor' 'cheap' 'bulky' 'anticipated' 'maternity'
 'disappointing' 'awkward' 'huge' 'covered']


CLASSE 0
Les dix variables ayant l'association négative la plus forte avec la classe 0 :
['waistline' 'wound' 'allows' 'virtually' 'friday' 'preference' 'hi'
 'customer' 'holding' 'waiting']

Les dix variables ayant l'association positive la plus forte avec la classe 0 :
['replacement' 'prior' 'patterned' 'hadn' 'ddd' 'continue' 'pulls' 'bc'
 'scrunch' 'fair']


CLASSE 1
Les dix variables ayant l'association négative la plus forte avec la classe 1 :
['returning' 'disappointing' 'odd' 'huge' 'waste' 'disappointed' 'poor'
 'idea' 'apart' 'unfortunately']

Les dix variables ayant l'association positive la plus forte

COMMENTAIRE A METTRE

In [109]:
model_lr = LogisticRegression(multi_class='multinomial',
                              solver='lbfgs').fit(X_train_vectorized_tfidf, y_train)
predictions_valid = model_lr.predict(X_valid_vectorized_tfidf)

c:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [110]:
accuracy_score(y_valid, predictions_valid)

0.7223728813559323

In [111]:
print(classification_report(y_valid, predictions_valid))

              precision    recall  f1-score   support

          -1       0.64      0.28      0.39       312
           0       0.61      0.61      0.61      1019
           1       0.79      0.88      0.83      1619

    accuracy                           0.72      2950
   macro avg       0.68      0.59      0.61      2950
weighted avg       0.71      0.72      0.71      2950



In [112]:
feature_names = np.array(vect_tfidf.get_feature_names_out())
idx_tfidf_sorted = X_train_vectorized_tfidf.max(0).toarray()[0].argsort()
print("TF-IDF le moins élevé : {}".format(feature_names[idx_tfidf_sorted[:10]]))
print("TF-IDF le plus élevé : {}".format(feature_names[idx_tfidf_sorted[:-11:-1]]))

TF-IDF le moins élevé : ['shut' 'pros' 'secondly' 'wondering' 'fo' 'wi' 'trigger' 'lik' 'framed'
 'monitor']
TF-IDF le plus élevé : ['birds' 'structure' 'amp' 'comfort' 'simple' 'wonderful' 'exciting' 'she'
 'royal' 'awesome']


Nous faisons avec les mêmes paramètres mais avec le vectoriseur à unigrammes et bigrammes.

In [113]:
model_lr = LogisticRegression(multi_class='multinomial', solver='lbfgs',max_iter=500).fit(X_train_vectorized_count_bigrams, y_train)
predictions_valid = model_lr.predict(X_valid_vectorized_count_bigrams)

In [114]:
accuracy_score(y_valid, predictions_valid)

0.6983050847457627

In [115]:
print(classification_report(y_valid, predictions_valid))

              precision    recall  f1-score   support

          -1       0.52      0.39      0.44       312
           0       0.58      0.58      0.58      1019
           1       0.79      0.84      0.81      1619

    accuracy                           0.70      2950
   macro avg       0.63      0.60      0.61      2950
weighted avg       0.69      0.70      0.69      2950



In [116]:
print_n_strongly_associated_features(vect_count_bigrams, model_lr, 10)

CLASSE -1
Les dix variables ayant l'association négative la plus forte avec la classe -1 :
['soft' 'amazing' 'compliments' 'love' 'think' 'little' 'beautiful'
 'great' 'comfortable' 'be too']

Les dix variables ayant l'association positive la plus forte avec la classe -1 :
['cheap' 'unflattering' 'huge' 'frumpy' 'to love' 'were' 'going back'
 'poor' 'bulky' 'weird']


CLASSE 0
Les dix variables ayant l'association négative la plus forte avec la classe 0 :
['spring summer' 'but this' 'is lightweight' 'much fabric' 'sweater the'
 'and didn' 'ever' 'elastic waist' 'bought size' 'more than']

Les dix variables ayant l'association positive la plus forte avec la classe 0 :
['top for' 'issue' 'not flattering' 'good for' 'is cute' 'returning it'
 'this fit' 'design but' 'easily' 'is too']


CLASSE 1
Les dix variables ayant l'association négative la plus forte avec la classe 1 :
['not flattering' 'returning' 'to love' 'disappointed' 'huge' 'were'
 'is too' 'unfortunately' 'going back' 'scratchy

### SVM

In [117]:
from sklearn.svm import SVC

In [118]:
model_svm = SVC(kernel='linear', C=0.1).fit(X_train_vectorized_count_bigrams, y_train)
predictions_valid = model_svm.predict(X_valid_vectorized_count_bigrams)

In [119]:
accuracy_score(y_valid, predictions_valid)

0.6874576271186441

In [120]:
print(classification_report(y_valid, predictions_valid))

              precision    recall  f1-score   support

          -1       0.46      0.44      0.45       312
           0       0.57      0.55      0.56      1019
           1       0.79      0.82      0.81      1619

    accuracy                           0.69      2950
   macro avg       0.61      0.60      0.61      2950
weighted avg       0.68      0.69      0.68      2950

